# SQL queries - Lab


## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [8]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [9]:
# Your code here; create the connection
conn=sqlite3.connect('data.sqlite')

#### Find the number of customers served by  employees

In [20]:
query="""SELECT e.employeeNumber,e.lastName||''||e.firstname as name,
COUNT(DISTINCT c.customerNumber) AS num_customers_served
FROM customers as c
JOIN employees as e
ON c.salesRepEmployeeNumber=e.employeeNumber
GROUP BY e.employeeNumber,name
ORDER BY num_customers_served DESC
LIMIT 10

"""
pd.read_sql(query,conn)

,employeeNumber,name,num_customers_served
0,1401,CastilloPamela,10
1,1504,JonesBarry,9
2,1323,VanaufGeorge,8
3,1501,BottLarry,8
4,1286,TsengFoon Yue,7
5,1370,HernandezGerard,7
6,1165,JenningsLeslie,6
7,1166,ThompsonLeslie,6
8,1188,FirrelliJulie,6
9,1216,PattersonSteve,6


In [24]:
query="""SELECT 
orderNumber,
productName,
COUNT(quantityOrdered) as num_ordered
FROM orderdetails
JOIN products
USING(productCode)
GROUP BY productName,orderNumber
HAVING AVG(quantityOrdered)>20
ORDER BY num_ordered asc
LIMIT 5"""
pd.read_sql(query,conn)

,orderNumber,productName,num_ordered
0,10103,18th Century Vintage Horse Carriage,1
1,10111,18th Century Vintage Horse Carriage,1
2,10126,18th Century Vintage Horse Carriage,1
3,10150,18th Century Vintage Horse Carriage,1
4,10163,18th Century Vintage Horse Carriage,1


## Window functions
`RANK()`, `DENSE_RANK()`, and `ROW_NUMBER()` window functions are used in SQL.

`RANK()`-gives the same rank for tied values,leaves gaps.

`DENSE_RANK()`-gives the same rank for tied values without gaps.

`ROW_NUMBER()`-gives consecutive number without gaps.

#### Using CTE with window functions to find the highest amount paid by customers

In [7]:
query=""" WITH cust as (SELECT
customerName,
city,
amount,
ROW_NUMBER() OVER (PARTITION BY customerName ORDER BY amount desc) as row_num
FROM customers
JOIN payments
USING(customerNumber)
LIMIT 10)
SELECT 
customerName,
city,
amount
FROM cust
WHERE row_num=1
"""
pd.read_sql(query,conn)
#partition by clause partitions the data based on the specified column

,customerName,city,amount
0,Diecast Classics Inc.,Allentown,63357.13
1,GiftsForHim.com,Auckland,36442.34
2,"Down Under Souveniers, Inc",Auckland,75020.13


### Rank products by buyprice

In [12]:
query="""SELECT
productName,
buyPrice,
RANK () OVER(ORDER BY buyPrice ASC) AS rank
FROM products
LIMIT 5
"""
pd.read_sql(query,conn)

,productName,buyPrice,rank
0,1958 Chevy Corvette Limited Edition,15.91,1
1,1982 Lamborghini Diablo,16.24,2
2,1938 Cadillac V-16 Presidential Limousine,20.61,3
3,1936 Mercedes Benz 500k Roadster,21.75,4
4,1939 Chevrolet Deluxe Coupe,22.57,5


### Calculate moving total of `priceEach `

In [16]:
query="""SELECT 
orderDate, 
priceEach,
SUM(priceEach) OVER( ORDER BY orderDate ROWS BETWEEN 2 PRECEDING  AND CURRENT ROW) AS moving_total
FROM orders
JOIN orderdetails
USING(orderNumber)
LIMIT 5"""
pd.read_sql(query,conn)

,orderDate,priceEach,moving_total
0,2003-01-06,35.29,35.29
1,2003-01-06,55.09,90.38
2,2003-01-06,75.46,165.84
3,2003-01-06,136.00,266.55
4,2003-01-09,32.53,243.99


## Select last names and first names of the top 10 customers who have placed orders with a status of Shipped


In [3]:
# Your code here
query="""SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE status = 'Shipped'
LIMIT 10
"""
pd.read_sql(query,conn)

,customerNumber,contactLastName,contactFirstName
0,363,Young,Dorothy
1,128,Keitel,Roland
2,181,Frick,Michael
3,121,Bergulfsen,Jonas
4,141,Freyre,Diego
5,145,Petersen,Jytte
6,278,Rovelli,Giovanni
7,131,Lee,Kwai
8,385,Cruz,Arnold
9,486,Salazar,Rosa


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
query="""SELECT 
productName, 
count(orderNumber) as total_orders
FROM products
JOIN orderdetails
USING(productCode)
GROUP BY productName
ORDER BY total_orders DESC
LIMIT 10"""
pd.read_sql(query,conn)

,productName,total_orders
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
5,Corsair F4U ( Bird Cage),28
6,Boeing X-32A JSF,28
7,American Airlines: MD-11S,28
8,American Airlines: B767-300,28
9,America West Airlines B757-200,28


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
query="""SELECT 
productName,
COUNT(DISTINCT customerNumber) as total_people
FROM customers
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
JOIN products
USING(productCode)
GROUP BY productName
ORDER BY total_people desc
LIMIT 5"""
pd.read_sql(query,conn)

,productName,total_people
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [6]:
# Your code here
query="""SELECT 
e.employeeNumber,
e.lastName,
e.firstName,
COUNT(c.customerNumber) as num_customers
FROM employees as e
JOIN customers as c
ON e.employeeNumber=c.salesRepEmployeeNumber
GROUP BY e.employeeNumber,e.lastName,e.firstName
HAVING AVG(c.creditLimit)>15000
"""
pd.read_sql(query,conn)

,employeeNumber,lastName,firstName,num_customers
0,1165,Jennings,Leslie,6
1,1166,Thompson,Leslie,6
2,1188,Firrelli,Julie,6
3,1216,Patterson,Steve,6
4,1286,Tseng,Foon Yue,7
5,1323,Vanauf,George,8
6,1337,Bondur,Loui,6
7,1370,Hernandez,Gerard,7
8,1401,Castillo,Pamela,10
9,1501,Bott,Larry,8


In [14]:
#Select number of employees in each city with diferent jobtitles
query=""" SELECT 
e.jobTitle,
o.city,
COUNT(e.employeeNumber) as num_employees
FROM employees as e
JOIN offices as o
ON e.officeCode=o.officeCode
GROUP BY e.jobTitle,o.city
ORDER BY num_employees DESC

LIMIT 5"""
pd.read_sql(query,conn)

,jobTitle,city,num_employees
0,Sales Rep,Paris,4
1,Sales Rep,Sydney,3
2,Sales Rep,Boston,2
3,Sales Rep,London,2
4,Sales Rep,NYC,2


In [19]:
query="""SELECT 
employeeNumber,
(lastName||' '||firstName) AS name,
jobTitle
FROM employees
WHERE jobTitle='Sales Rep'
ORDER BY employeeNumber DESC
"""
pd.read_sql(query,conn)

,employeeNumber,name,jobTitle
0,1702,Gerard Martin,Sales Rep
1,1625,Kato Yoshimi,Sales Rep
2,1621,Nishi Mami,Sales Rep
3,1619,King Tom,Sales Rep
4,1612,Marsh Peter,Sales Rep
5,1611,Fixter Andy,Sales Rep
6,1504,Jones Barry,Sales Rep
7,1501,Bott Larry,Sales Rep
8,1401,Castillo Pamela,Sales Rep
9,1370,Hernandez Gerard,Sales Rep


In [26]:
query="""SELECT  
customerNumber,
customerName,
city,
creditLimit,
ROW_NUMBER () OVER (PARTITION BY customerNumber ORDER BY creditLimit DESC) AS row_num
FROM customers
LIMIT 5"""
pd.read_sql(query,conn)

,customerNumber,customerName,city,creditLimit,row_num
0,103,Atelier graphique,Nantes,21000,1
1,112,Signal Gift Stores,Las Vegas,71800,1
2,114,"Australian Collectors, Co.",Melbourne,117300,1
3,119,La Rochelle Gifts,Nantes,118200,1
4,121,Baane Mini Imports,Stavern,81700,1
